In [1]:
import json
import datetime
from contextlib import ExitStack

In [2]:
import abc

class EntryProcessor:

    __metaclass__ = abc.ABCMeta

    def __init__(self, output_file: str):
        self.file_path = output_file

    def __enter__(self, *args):
        self.file_handle = open(self.file_path, "w")

    def __exit__(self, *args):
        self.file_handle.close()

    @abc.abstractmethod
    def can_handle_entry(self, json_entry):
        return False

    @abc.abstractmethod
    def produce_output_lines(self, parsed_entry):
        yield

    @abc.abstractmethod
    def write_header(self):
        return

    def format_field(self, payload_entry:dict, key:str, quotes=True, can_be_missing=False):
        if can_be_missing and key not in payload_entry:
            return ""

        if quotes:
            return f'"{payload_entry[key]}"'
        else:
            return payload_entry[key]

    def format_date(self, date_utc : datetime.datetime) -> str:
        date = date_utc + datetime.timedelta(hours=-3)
        return date.strftime("%d/%m/%Y")

    def write_entry(self, parsed_entry):
        for line in self.produce_output_lines(parsed_entry):
            self.file_handle.write("\n")
            self.file_handle.write(line)



class CuratedOfferOptionsProcessor(EntryProcessor):

    def __init__(self, output_file: str):
        super().__init__(output_file)

        self.header = ["CurationProvider", "OfferId", "DealerId", "UniqueOptionId", "OptionId", "IsMobileDealer", "IsOpen", "Eta", "ChamaScore", "ProductBrand", "IsWinner", "MinimumPrice", "MaximumPrice", "DynamicPrice", "FinalPrice", "DefeatPrimaryReason", "DefeatReasons", "EnqueuedTimeSP"]

    def can_handle_entry(self, parsed_entry):
        return parsed_entry["EventName"] == "CurateOffer_Result"

    def write_header(self):
        self.file_handle.write(",".join(self.header))

    def produce_output_lines(self, parsed_entry):
        for payload_entry in parsed_entry["Payload"]:
            output_dict = {}
            options = payload_entry["options"]
            for option_entry in options:
                output_dict["CurationProvider"] = self.format_field(payload_entry, "curationProvider", True)
                output_dict["OfferId"] = self.format_field(payload_entry, "offerId", True)
                output_dict["DealerId"] = self.format_field(payload_entry, "dealerId", True)
                output_dict["UniqueOptionId"] = self.format_field(option_entry, "uniqueOptionId", True)
                output_dict["OptionId"] = self.format_field(option_entry, "optionId", True)
                output_dict["IsMobileDealer"] = self.format_field(option_entry, "isMobileDealer", True)
                output_dict["IsOpen"] = self.format_field(option_entry, "isOpen", False)
                output_dict["Eta"] = self.format_field(option_entry, "eta", True)
                output_dict["ChamaScore"] = self.format_field(option_entry, "chamaScore", False)
                output_dict["ProductBrand"] = self.format_field(option_entry, "productBrand", True)
                output_dict["IsWinner"] = self.format_field(option_entry, "isWinner", False)
                output_dict["MinimumPrice"] = self.format_field(option_entry, "minimumPrice", False)
                output_dict["MaximumPrice"] = self.format_field(option_entry, "maximumPrice", False)
                output_dict["DynamicPrice"] = self.format_field(option_entry, "dynamicPrice", False)
                output_dict["FinalPrice"] = self.format_field(option_entry, "finalPrice", False)
                output_dict["DefeatPrimaryReason"] = self.format_field(option_entry, "defeatPrimaryReason", True, can_be_missing=True)
                output_dict["DefeatReasons"] = self.format_field(option_entry, "defeatReasons", True, can_be_missing=True)
                output_dict["EnqueuedTimeSP"] = self.format_date(parsed_entry["EnqueuedTimeUtc"])
                yield ",".join([str(output_dict[key]) for key in self.header])

class DynamicPriceOptionProcessor(EntryProcessor):

    def __init__(self, output_file: str):
        super().__init__(output_file)
        self.header = ["Provider", "OfferId", "UniqueOptionId", "BestPrice", "EnqueuedTimeSP"]

    def can_handle_entry(self, parsed_entry):
        if parsed_entry["EventName"] != "DynamicPrice_Result":
            return False
        return parsed_entry["Payload"]["provider"] == "ApplyDynamicPricePerOption"

    def write_header(self):
        self.file_handle.write(",".join(self.header))

    def produce_output_lines(self, parsed_entry):
        output_dict = {}
        payload_entry = parsed_entry["Payload"]
        algorithm_output = payload_entry["algorithmOutput"]

        for algorithm_output_entry in algorithm_output:
            output_dict["Provider"] = self.format_field(payload_entry, "provider", True)
            output_dict["OfferId"] = self.format_field(payload_entry, "offerId", True)
            output_dict["UniqueOptionId"] = self.format_field(algorithm_output_entry, "uniqueOptionId", True)
            output_dict["BestPrice"] = self.format_field(algorithm_output_entry, "bestPrice", False)
            output_dict["EnqueuedTimeSP"] = self.format_date(parsed_entry["EnqueuedTimeUtc"])
            yield ",".join([str(output_dict[key]) for key in self.header])


class DynamicPriceRangeProcessor(EntryProcessor):

    def __init__(self, output_file: str):
        super().__init__(output_file)
        self.header = ["Provider", "OfferId", "MinGlobal", "MinRecommended", "MaxRecommended", "DifferenceMinRecommendMinTheory", "EnqueuedTimeSP"]

    def can_handle_entry(self, parsed_entry):
        if parsed_entry["EventName"] != "DynamicPrice_Result":
            return False
        return parsed_entry["Payload"]["provider"] == "ApplyDynamicPriceRange"

    def write_header(self):
        self.file_handle.write(",".join(self.header))

    def produce_output_lines(self, parsed_entry):
        output_dict = {}
        payload_entry = parsed_entry["Payload"]
        output_dict["Provider"] = self.format_field(payload_entry, "provider", True)
        output_dict["OfferId"] = self.format_field(payload_entry, "offerId", True)

        algorithm_output = payload_entry["algorithmOutput"]
        output_dict["MinGlobal"] = self.format_field(algorithm_output, "min_global", False)
        output_dict["MinRecommended"] = self.format_field(algorithm_output, "min_recommended", False)
        output_dict["MaxRecommended"] = self.format_field(algorithm_output, "max_recommended", False)
        output_dict["DifferenceMinRecommendMinTheory"] = self.format_field(algorithm_output, "differenceMinRecommendMinTheory", False)
        output_dict["EnqueuedTimeSP"] = self.format_date(parsed_entry["EnqueuedTimeUtc"])
        yield ",".join([str(output_dict[key]) for key in self.header])

In [3]:
class Parser:

    def __init__(self):
        pass

    def parse_file(self, input_file: str):
        with open(input_file) as f:
            loaded_data = json.load(f)
            for entry in loaded_data:
                yield self.parse_entry(entry)

    def parse_date_utc(self, date_string : str) -> datetime.datetime:
        return datetime.datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S %Z')

    def parse_entry(self, json_entry: dict):
        json_entry["Payload"] = json.loads(json_entry["Payload"])
        json_entry["EnqueuedTimeUtc"] = self.parse_date_utc(json_entry["EnqueuedTimeUtc"])
        return json_entry

In [4]:
writers = [CuratedOfferOptionsProcessor("CuratedOfferOptions.csv"),
           DynamicPriceOptionProcessor("DynamicPriceOption.csv"),
           DynamicPriceRangeProcessor("DynamicPriceRange.csv")]

with ExitStack() as stack:
    for writer in writers:
        stack.enter_context(writer)

    data = Parser().parse_file("case.json")

    for writer in writers:
        writer.write_header()

    for entry in data:
        for writer in writers:
            if writer.can_handle_entry(entry):
                writer.write_entry(entry)
                break